In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import random

# Load dataset
data = pd.read_csv('heart_failure_clinical_records_dataset.csv')
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Standardize dataset
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define Particle Swarm Optimization (PSO) parameters
SWARM_SIZE = 10
GENERATIONS = 5
W = 0.5  # Inertia weight
C1 = 1.5  # Cognitive (personal) weight
C2 = 1.5  # Social (group) weight

def evaluate_model(individual):
    """Evaluate a model based on an individual's parameters."""
    model = Sequential([
        Dense(individual[0], input_dim=X_train.shape[1], activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(individual[1]),
        Dense(individual[2], activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(individual[3]),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=individual[4]), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=20, batch_size=individual[5], verbose=0, validation_split=0.2)
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy

def initialize_particles():
    """Generate an initial swarm of particles."""
    particles = [
        [random.randint(10, 100),  # First layer neurons
         random.uniform(0.1, 0.5),  # First layer dropout
         random.randint(10, 100),  # Second layer neurons
         random.uniform(0.1, 0.5),  # Second layer dropout
         random.uniform(0.0001, 0.01),  # Learning rate
         random.randint(16, 128)]  # Batch size
        for _ in range(SWARM_SIZE)
    ]
    velocities = [
        [random.uniform(-1, 1) for _ in range(len(particles[0]))]
        for _ in range(SWARM_SIZE)
    ]
    return particles, velocities

# PSO optimization
particles, velocities = initialize_particles()
personal_best = particles.copy()
global_best = max(particles, key=evaluate_model)
personal_best_scores = [evaluate_model(p) for p in particles]

def update_velocity(velocity, particle, personal_best, global_best):
    """Update the velocity of a particle."""
    new_velocity = []
    for i in range(len(particle)):
        r1, r2 = random.random(), random.random()
        cognitive = C1 * r1 * (personal_best[i] - particle[i])
        social = C2 * r2 * (global_best[i] - particle[i])
        new_velocity.append(W * velocity[i] + cognitive + social)
    return new_velocity

def update_particle(particle, velocity):
    """Update the position of a particle."""
    new_particle = [
        max(10, min(100, int(particle[0] + velocity[0]))),
        max(0.1, min(0.5, particle[1] + velocity[1])),
        max(10, min(100, int(particle[2] + velocity[2]))),
        max(0.1, min(0.5, particle[3] + velocity[3])),
        max(0.0001, min(0.01, particle[4] + velocity[4])),
        max(16, min(128, int(particle[5] + velocity[5])))
    ]
    return new_particle

for generation in range(GENERATIONS):
    for i in range(SWARM_SIZE):
        fitness = evaluate_model(particles[i])
        if fitness > personal_best_scores[i]:
            personal_best_scores[i] = fitness
            personal_best[i] = particles[i]
        if fitness > evaluate_model(global_best):
            global_best = particles[i]
        velocities[i] = update_velocity(velocities[i], particles[i], personal_best[i], global_best)
        particles[i] = update_particle(particles[i], velocities[i])

    print(f"Generation {generation + 1}, Best Accuracy: {evaluate_model(global_best)}")

# Best solution
best_individual = global_best
print("Best Individual:", best_individual)

# Train the best model
best_model = Sequential([
    Dense(best_individual[0], input_dim=X_train.shape[1], activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(best_individual[1]),
    Dense(best_individual[2], activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(best_individual[3]),
    Dense(1, activation='sigmoid')
])
best_model.compile(optimizer=Adam(learning_rate=best_individual[4]), loss='binary_crossentropy', metrics=['accuracy'])
best_model.fit(X_train, y_train, epochs=20, batch_size=best_individual[5], verbose=0)

# Extract features for Random Forest
train_features = best_model.predict(X_train)
test_features = best_model.predict(X_test)

# Train Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(train_features, y_train)

# Predictions and classification report
rf_predictions = rf_model.predict(test_features)
print("Classification Report:\n", classification_report(y_test, rf_predictions))

2025-01-09 02:44:17.990151: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-09 02:44:17.994090: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-09 02:44:18.007712: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736370858.030584   25168 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736370858.037396   25168 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-09 02:44:18.059742: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

Generation 1, Best Accuracy: 0.7666666507720947
Generation 2, Best Accuracy: 0.75
Generation 3, Best Accuracy: 0.800000011920929
Generation 4, Best Accuracy: 0.75
Generation 5, Best Accuracy: 0.7666666507720947
Best Individual: [30, 0.5, 15, 0.4389933370184338, 0.01, 85]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.91      0.81        35
           1       0.81      0.52      0.63        25

    accuracy                           0.75        60
   macro avg       0.77      0.72      0.72        60
weighted avg       0.76      0.75      0.74        60



In [2]:
import tensorflow as tf
print(tf.__version__)


2.18.0


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

model = Sequential([
    Input(shape=(input_dim,)),  # Specify the input shape here
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])


2025-01-07 02:12:58.059696: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-07 02:12:58.063093: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-07 02:12:58.075840: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736196178.094819  609339 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736196178.099569  609339 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-07 02:12:58.115127: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [5]:
input_dim = 11  # Number of features in your dataset


In [3]:
x=10
y=10
z=x+y
print(z)

20


In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import random

# Load dataset
data = pd.read_csv('heart_failure_clinical_records_dataset.csv')
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Standardize dataset
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Genetic Algorithm Parameters
POPULATION_SIZE = 20
GENERATIONS = 10
MUTATION_RATE = 0.1

def evaluate_model(individual):
    """Evaluate a model based on an individual's parameters."""
    model = Sequential([
        Dense(individual[0], input_dim=X_train.shape[1], activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(individual[1]),
        Dense(individual[2], activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(individual[3]),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=individual[4]), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=20, batch_size=individual[5], verbose=0, validation_split=0.2)
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy

def initialize_population():
    """Initialize the population with random individuals."""
    population = [
        [random.randint(10, 100),  # First layer neurons
         random.uniform(0.1, 0.5),  # First layer dropout
         random.randint(10, 100),  # Second layer neurons
         random.uniform(0.1, 0.5),  # Second layer dropout
         random.uniform(0.0001, 0.01),  # Learning rate
         random.randint(16, 128)]  # Batch size
        for _ in range(POPULATION_SIZE)
    ]
    return population

def crossover(parent1, parent2):
    """Perform crossover between two parents."""
    point = random.randint(1, len(parent1) - 2)  # Random crossover point
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def mutate(individual):
    """Perform mutation on an individual."""
    if random.random() < MUTATION_RATE:
        index = random.randint(0, len(individual) - 1)
        if index in [0, 2, 5]:  # Integer values
            individual[index] = random.randint(10, 100 if index != 5 else 128)
        elif index in [1, 3]:  # Float values for dropout
            individual[index] = random.uniform(0.1, 0.5)
        elif index == 4:  # Float values for learning rate
            individual[index] = random.uniform(0.0001, 0.01)
    return individual

# Initialize population
population = initialize_population()
best_individual = None
best_score = -np.inf

for generation in range(GENERATIONS):
    # Evaluate fitness
    fitness_scores = [evaluate_model(individual) for individual in population]
    
    # Update the best individual
    for i, score in enumerate(fitness_scores):
        if score > best_score:
            best_score = score
            best_individual = population[i]
    
    # Select parents based on fitness (tournament selection)
    selected_parents = []
    for _ in range(POPULATION_SIZE // 2):
        parent1, parent2 = random.choices(population, weights=fitness_scores, k=2)
        selected_parents.append((parent1, parent2))
    
    # Generate next population through crossover and mutation
    next_population = []
    for parent1, parent2 in selected_parents:
        child1, child2 = crossover(parent1, parent2)
        next_population.extend([mutate(child1), mutate(child2)])
    
    population = next_population[:POPULATION_SIZE]  # Ensure population size remains constant
    print(f"Generation {generation + 1}, Best Accuracy: {best_score}")

# Best solution
print("Best Individual:", best_individual)

# Train the best model
best_model = Sequential([
    Dense(best_individual[0], input_dim=X_train.shape[1], activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(best_individual[1]),
    Dense(best_individual[2], activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(best_individual[3]),
    Dense(1, activation='sigmoid')
])
best_model.compile(optimizer=Adam(learning_rate=best_individual[4]), loss='binary_crossentropy', metrics=['accuracy'])
best_model.fit(X_train, y_train, epochs=20, batch_size=best_individual[5], verbose=0)

# Extract features for Random Forest
train_features = best_model.predict(X_train)
test_features = best_model.predict(X_test)

# Train Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(train_features, y_train)

# Predictions and classification report
rf_predictions = rf_model.predict(test_features)
print("Classification Report:\n", classification_report(y_test, rf_predictions))


/home/vanamabhinav/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Generation 1, Best Accuracy: 0.8166666626930237
Generation 2, Best Accuracy: 0.8166666626930237
Generation 3, Best Accuracy: 0.8166666626930237
Generation 4, Best Accuracy: 0.8166666626930237
Generation 5, Best Accuracy: 0.8166666626930237
Generation 6, Best Accuracy: 0.8166666626930237
Generation 7, Best Accuracy: 0.8166666626930237
Generation 8, Best Accuracy: 0.8166666626930237
Generation 9, Best Accuracy: 0.8166666626930237
Generation 10, Best Accuracy: 0.8166666626930237
Best Individual: [69, 0.25887004795805824, 78, 0.27533479254857013, 0.0024852601360624274, 93]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.94      0.81        35
           1       0.86      0.48      0.62        25

    accuracy                           0.75        60
   macro avg       0.79      0.71      0.72        60
weighted avg       0.78      0.75      0.73        6

In [8]:
import matplotlib.pyplot as plt

# Initialize variables to store global best accuracy per generation
global_best_accuracies = []

for generation in range(10):
    for i in range(10):
        fitness = evaluate_model(particles[i])
        if fitness > personal_best_scores[i]:
            personal_best_scores[i] = fitness
            personal_best[i] = particles[i]
        if fitness > evaluate_model(global_best):
            global_best = particles[i]
        velocities[i] = update_velocity(velocities[i], particles[i], personal_best[i], global_best)
        particles[i] = update_particle(particles[i], velocities[i])

    # Track the global best accuracy for the current generation
    best_accuracy = evaluate_model(global_best)
    global_best_accuracies.append(best_accuracy)
    print(f"Generation {generation + 1}, Best Accuracy: {best_accuracy}")

# Plot the global best accuracy over generations
plt.figure(figsize=(10, 6))
plt.plot(range(1, GENERATIONS + 1), global_best_accuracies, marker='o', linestyle='-', color='b')
plt.title("Global Best Accuracy Over Generations", fontsize=16)
plt.xlabel("Generation", fontsize=14)
plt.ylabel("Accuracy", fontsize=14)
plt.grid(True)
plt.show()


NameError: name 'evaluate_model' is not defined

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import joblib
import random

# Load dataset
data = pd.read_csv('heart_failure_clinical_records_dataset.csv')  # Update with your dataset
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Standardize dataset
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Genetic Algorithm Parameters
POPULATION_SIZE = 20
GENERATIONS = 10
MUTATION_RATE = 0.1

def evaluate_model(individual):
    """Evaluate a model based on an individual's parameters."""
    model = Sequential([
        Dense(individual[0], input_dim=X_train.shape[1], activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(individual[1]),
        Dense(individual[2], activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(individual[3]),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=individual[4]), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=20, batch_size=individual[5], verbose=0, validation_split=0.2)
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy

def initialize_population():
    """Initialize the population with random individuals."""
    population = [
        [random.randint(10, 100),  # First layer neurons
         random.uniform(0.1, 0.5),  # First layer dropout
         random.randint(10, 100),  # Second layer neurons
         random.uniform(0.1, 0.5),  # Second layer dropout
         random.uniform(0.0001, 0.01),  # Learning rate
         random.randint(16, 128)]  # Batch size
        for _ in range(POPULATION_SIZE)
    ]
    return population

def crossover(parent1, parent2):
    """Perform crossover between two parents."""
    point = random.randint(1, len(parent1) - 2)  # Random crossover point
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def mutate(individual):
    """Perform mutation on an individual."""
    if random.random() < MUTATION_RATE:
        index = random.randint(0, len(individual) - 1)
        if index in [0, 2, 5]:  # Integer values
            individual[index] = random.randint(10, 100 if index != 5 else 128)
        elif index in [1, 3]:  # Float values for dropout
            individual[index] = random.uniform(0.1, 0.5)
        elif index == 4:  # Float values for learning rate
            individual[index] = random.uniform(0.0001, 0.01)
    return individual

# Initialize population
population = initialize_population()
best_individual = None
best_score = -np.inf

for generation in range(GENERATIONS):
    # Evaluate fitness
    fitness_scores = [evaluate_model(individual) for individual in population]
    
    # Update the best individual
    for i, score in enumerate(fitness_scores):
        if score > best_score:
            best_score = score
            best_individual = population[i]
    
    # Select parents based on fitness (tournament selection)
    selected_parents = []
    for _ in range(POPULATION_SIZE // 2):
        parent1, parent2 = random.choices(population, weights=fitness_scores, k=2)
        selected_parents.append((parent1, parent2))
    
    # Generate next population through crossover and mutation
    next_population = []
    for parent1, parent2 in selected_parents:
        child1, child2 = crossover(parent1, parent2)
        next_population.extend([mutate(child1), mutate(child2)])
    
    population = next_population[:POPULATION_SIZE]  # Ensure population size remains constant
    print(f"Generation {generation + 1}, Best Accuracy: {best_score}")

# Best solution found
print("Best Individual:", best_individual)

# Train the best model
heart_stroke_model = Sequential([
    Dense(best_individual[0], input_dim=X_train.shape[1], activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(best_individual[1]),
    Dense(best_individual[2], activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(best_individual[3]),
    Dense(1, activation='sigmoid')
])
heart_stroke_model.compile(optimizer=Adam(learning_rate=best_individual[4]), loss='binary_crossentropy', metrics=['accuracy'])
heart_stroke_model.fit(X_train, y_train, epochs=20, batch_size=best_individual[5], verbose=0)

# Save the trained model
heart_stroke_model.save("heart_stroke_model.h5")
print("Heart Stroke Model Saved!")

# Extract features for Random Forest
train_features = heart_stroke_model.predict(X_train)
test_features = heart_stroke_model.predict(X_test)

# Train Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(train_features, y_train)

# Save Random Forest Model
joblib.dump(rf_model, "random_forest_model.pkl")
print("Random Forest Model Saved!")

# Predictions and classification report
rf_predictions = rf_model.predict(test_features)
print("Classification Report:\n", classification_report(y_test, rf_predictions))


2025-01-29 10:31:19.032540: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-29 10:31:19.186715: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-29 10:31:19.458542: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738126879.702434    6186 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738126879.765191    6186 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-29 10:31:20.459039: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

Generation 1, Best Accuracy: 0.800000011920929
Generation 2, Best Accuracy: 0.800000011920929
Generation 3, Best Accuracy: 0.8166666626930237
Generation 4, Best Accuracy: 0.8166666626930237
Generation 5, Best Accuracy: 0.8166666626930237
Generation 6, Best Accuracy: 0.8166666626930237
Generation 7, Best Accuracy: 0.8166666626930237
Generation 8, Best Accuracy: 0.8166666626930237
Generation 9, Best Accuracy: 0.8166666626930237
Generation 10, Best Accuracy: 0.8166666626930237
Best Individual: [92, 0.28831034030097713, 38, 0.194902292213839, 0.0032690074324506838, 31]


Heart Stroke Model Saved!
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Random Forest Model Saved!
Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.91      0.81        35
           1       0.81      0.52      0.63        25

    accuracy                           0.75        60
   macro avg       0.77      0.72      0.72        60
weighted avg       0.76      0.75      0.74        60

